In [6]:
import pyspark
from pyspark.sql.types import *

import numpy as np
import pandas as pd

from datetime import *
from dateutil.parser import parse

from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
output_notebook()

from bokeh.charts import Bar, output_file, show

sql = SQLContext(sc)

Loading BokehJS ...

In [7]:
data_path = "../input/csv/"
match_rdd = sc.textFile(data_path + "matches.csv")

# header = match_rdd.first()
# fields = [StructField(field_name, StringType(), False) for field_name in header.split(',')]
# fields[0].dataType = LongType()
# fields[3].dataType = DateType()
# fields[9].dataType = BooleanType()
# fields[11].dataType = LongType()
# fields[12].dataType = LongType()
# fields[17].nullable = True
# schema = StructType(fields)

match_header = match_rdd.filter(lambda l: "id,season" in l)
match_no_header = match_rdd.subtract(match_header)
match_temp_rdd = match_no_header.map(lambda k: k.split(',')).map(lambda p: (int(p[0]), p[1],p[2],parse(p[3]).date(),p[4],p[5],p[6],p[7],p[8],p[9]=='1',p[10],int(p[11]),int(p[12]),p[13],p[14],p[15],p[16],p[17]))
match_df = sql.createDataFrame(match_temp_rdd, match_rdd.first().split(','))
match_df = match_df.orderBy(match_df.id.asc())

In [8]:
overall_ranking = match_df.filter("season=2016").groupBy("winner").count().orderBy("count",ascending=0)
overall_ranking = overall_ranking.filter("winner != '' ")
overall_ranking.show(truncate=False)

+---------------------------+-----+
|winner                     |count|
+---------------------------+-----+
|Sunrisers Hyderabad        |11   |
|Royal Challengers Bangalore|9    |
|Gujarat Lions              |9    |
|Kolkata Knight Riders      |8    |
|Delhi Daredevils           |7    |
|Mumbai Indians             |7    |
|Rising Pune Supergiants    |5    |
|Kings XI Punjab            |4    |
+---------------------------+-----+



In [15]:
from bokeh.palettes import viridis

overall_pdf = overall_ranking.toPandas()
N = overall_ranking.count()
clr=viridis(N)

p = Bar(overall_pdf, values="count", label="winner", title="Overall Standings 2016", legend='bottom_right', plot_width=950, color=clr)

t = show(p, notebook_handle=True)